In [1]:
import cv2
import numpy as np
import mediapipe as mp
import joblib
from tensorflow.keras.models import load_model
from collections import deque
import math

# =============================================================================
# --- 1. KONFIGURASI DAN PEMUATAN MODEL ---
# =============================================================================

# -- Konfigurasi Umum --
# Threshold (Ambang Batas) untuk memicu peringatan
HEAD_DOWN_FRAMES_THRESHOLD = 20
EYES_CLOSED_FRAMES_THRESHOLD = 15
PITCH_THRESHOLD_DEG = 30 # Sudut kepala menunduk dalam derajat

# -- Model Pose Kepala --
try:
    HEAD_POSE_MODELS = {
        'pitch': joblib.load('model/headposeModel/xgb_pitch_model.joblib'),
        'yaw': joblib.load('model/headposeModel/xgb_yaw_model.joblib'),
        'roll': joblib.load('model/headposeModel/xgb_roll_model.joblib')
    }
    print("✅ Model Head Pose berhasil dimuat.")
except FileNotFoundError:
    print("❌ Error: File model Head Pose tidak ditemukan di direktori 'model/'.")
    exit()

# -- Model Status Mata --
try:
    EYE_STATUS_MODEL = load_model('model/eyeModel/eye_status_model.h5')
    print("✅ Model Status Mata berhasil dimuat.")
except (FileNotFoundError, IOError):
    print("❌ Error: File 'models/eye_status_model.h5' tidak ditemukan.")
    exit()
EYE_IMG_SIZE = 24
EYE_CLASS_LABELS = ['Mata Tertutup', 'Mata Terbuka']
LEFT_EYE_IDXS = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE_IDXS = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]


# -- Model Deteksi Menguap --
try:
    YAWN_SVM_MODEL = joblib.load('model/yawnModel/svm_yawn_detector.joblib')
    YAWN_SCALER = joblib.load('model/yawnModel/scaler.joblib')
    print("✅ Model Deteksi Menguap (SVM) berhasil dimuat.")
except FileNotFoundError:
    print("❌ Error: File 'svm_yawn_detector.joblib' atau 'scaler.joblib' tidak ditemukan.")
    exit()
MAR_WINDOW_SIZE = 20
MOUTH_INDICES = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291]

# -- Inisialisasi MediaPipe Face Mesh --
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True, # Diperlukan untuk landmark mata yang detail
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


# =============================================================================
# --- 2. FUNGSI-FUNGSI BANTUAN ---
# =============================================================================

# -- Fungsi untuk Pose Kepala --
def create_feature_vector(face_landmarks):
    anchor_point = face_landmarks.landmark[1]
    p_left = face_landmarks.landmark[359]
    p_right = face_landmarks.landmark[130]
    scale_distance = np.linalg.norm([p_left.x - p_right.x, p_left.y - p_right.y])
    if scale_distance < 1e-6: return None
    feature_vector = []
    for i in range(468):
        if i == 1: continue
        landmark = face_landmarks.landmark[i]
        feature_vector.extend([(landmark.x - anchor_point.x) / scale_distance, (landmark.y - anchor_point.y) / scale_distance, (landmark.z - anchor_point.z) / scale_distance])
    return np.array(feature_vector)

def draw_axes(img, pitch, yaw, roll, nose_2d, size=100):
    pitch_rad = pitch * np.pi / 180
    yaw_rad = -(yaw * np.pi / 180)
    roll_rad = roll * np.pi / 180
    Rx = np.array([[1, 0, 0], [0, math.cos(pitch_rad), -math.sin(pitch_rad)], [0, math.sin(pitch_rad), math.cos(pitch_rad)]])
    Ry = np.array([[math.cos(yaw_rad), 0, math.sin(yaw_rad)], [0, 1, 0], [-math.sin(yaw_rad), 0, math.cos(yaw_rad)]])
    Rz = np.array([[math.cos(roll_rad), -math.sin(roll_rad), 0], [math.sin(roll_rad), math.cos(roll_rad), 0], [0, 0, 1]])
    R = Rz @ Ry @ Rx
    axis = np.array([[size, 0, 0], [0, size, 0], [0, 0, size]])
    rotated_axis = R @ axis
    p1 = (int(nose_2d[0]), int(nose_2d[1]))
    p2_yaw = (int(nose_2d[0] + rotated_axis[0, 0]), int(nose_2d[1] + rotated_axis[1, 0])); cv2.line(img, p1, p2_yaw, (255, 0, 0), 3) # Yaw (Biru)
    p2_pitch = (int(nose_2d[0] + rotated_axis[0, 1]), int(nose_2d[1] + rotated_axis[1, 1])); cv2.line(img, p1, p2_pitch, (0, 255, 0), 3) # Pitch (Hijau)
    p2_roll = (int(nose_2d[0] + rotated_axis[0, 2]), int(nose_2d[1] + rotated_axis[1, 2])); cv2.line(img, p1, p2_roll, (0, 0, 255), 3) # Roll (Merah)
    return img

# -- Fungsi untuk Status Mata --
def preprocess_eye_for_predict(eye_img):
    gray_eye = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
    resized_eye = cv2.resize(gray_eye, (EYE_IMG_SIZE, EYE_IMG_SIZE))
    normalized_eye = resized_eye / 255.0
    input_eye = np.expand_dims(np.expand_dims(normalized_eye, axis=-1), axis=0)
    prediction = EYE_STATUS_MODEL.predict(input_eye, verbose=0)
    return prediction[0][0]

# -- Fungsi untuk Deteksi Menguap --
def calculate_mar(face_landmarks):
    p_vertical1 = face_landmarks.landmark[13]
    p_vertical2 = face_landmarks.landmark[14]
    p_horizontal1 = face_landmarks.landmark[78]
    p_horizontal2 = face_landmarks.landmark[308]
    vertical_dist = np.linalg.norm([p_vertical1.x - p_vertical2.x, p_vertical1.y - p_vertical2.y])
    horizontal_dist = np.linalg.norm([p_horizontal1.x - p_horizontal2.x, p_horizontal1.y - p_horizontal2.y])
    return vertical_dist / horizontal_dist if horizontal_dist > 0 else 0.0

# =============================================================================
# --- 3. EKSEKUSI UTAMA ---
# =============================================================================

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Error: Tidak bisa membuka kamera.")
        return
    print("🎥 Webcam berhasil dibuka. Tekan 'q' untuk keluar.")

    # Inisialisasi variabel status dan counter
    head_down_counter = 0
    eyes_closed_counter = 0
    mar_buffer = deque(maxlen=MAR_WINDOW_SIZE)
    yawn_status = "NORMAL"
    
    PITCH_THRESHOLD_RAD = -PITCH_THRESHOLD_DEG * np.pi / 180 # Konversi ke radian, negatif untuk menunduk

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break

        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        # Reset status per frame
        drowsiness_alert = False
        alert_reason = ""
        
        status_left = "Tidak Terdeteksi"
        status_right = "Tidak Terdeteksi"

        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0] # Ambil wajah pertama

            # --- A. Analisis Pose Kepala ---
            feature_vector = create_feature_vector(face_landmarks)
            if feature_vector is not None:
                feature_vector = feature_vector.reshape(1, -1)
                pitch_rad = HEAD_POSE_MODELS['pitch'].predict(feature_vector)[0]
                yaw_rad = HEAD_POSE_MODELS['yaw'].predict(feature_vector)[0]
                roll_rad = HEAD_POSE_MODELS['roll'].predict(feature_vector)[0]

                pitch_deg = pitch_rad * 180 / np.pi
                yaw_deg = yaw_rad * 180 / np.pi
                roll_deg = roll_rad * 180 / np.pi

                # Cek kondisi kepala menunduk
                if pitch_rad < PITCH_THRESHOLD_RAD:
                    head_down_counter += 1
                else:
                    head_down_counter = 0
                
                if head_down_counter >= HEAD_DOWN_FRAMES_THRESHOLD:
                    drowsiness_alert = True
                    alert_reason = "Kepala Terkulai"

                # Visualisasi pose
                nose_2d = (face_landmarks.landmark[1].x * w, face_landmarks.landmark[1].y * h)
                draw_axes(frame, pitch_deg, yaw_deg, roll_deg, nose_2d)
                cv2.putText(frame, f"Pitch: {pitch_deg:.1f}", (w - 150, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.putText(frame, f"Yaw: {yaw_deg:.1f}", (w - 150, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                cv2.putText(frame, f"Roll: {roll_deg:.1f}", (w - 150, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            # --- B. Analisis Status Mata ---
            padding = 5
            # Mata Kiri
            left_eye_points = np.array([[face_landmarks.landmark[i].x * w, face_landmarks.landmark[i].y * h] for i in LEFT_EYE_IDXS]).astype(int)
            lx_min, ly_min = np.min(left_eye_points, axis=0)
            lx_max, ly_max = np.max(left_eye_points, axis=0)
            if ly_max > ly_min and lx_max > lx_min:
                left_eye_crop = frame[ly_min-padding:ly_max+padding, lx_min-padding:lx_max+padding]
                if left_eye_crop.size > 0:
                    prob_left = preprocess_eye_for_predict(left_eye_crop)
                    status_left = EYE_CLASS_LABELS[1] if prob_left > 0.5 else EYE_CLASS_LABELS[0]
                    color = (0, 255, 0) if status_left == 'Mata Terbuka' else (0, 0, 255)
                    cv2.rectangle(frame, (lx_min-padding, ly_min-padding), (lx_max+padding, ly_max+padding), color, 2)
                    cv2.putText(frame, status_left, (lx_min-padding, ly_min-15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Mata Kanan
            right_eye_points = np.array([[face_landmarks.landmark[i].x * w, face_landmarks.landmark[i].y * h] for i in RIGHT_EYE_IDXS]).astype(int)
            rx_min, ry_min = np.min(right_eye_points, axis=0)
            rx_max, ry_max = np.max(right_eye_points, axis=0)
            if ry_max > ry_min and rx_max > rx_min:
                right_eye_crop = frame[ry_min-padding:ry_max+padding, rx_min-padding:rx_max+padding]
                if right_eye_crop.size > 0:
                    prob_right = preprocess_eye_for_predict(right_eye_crop)
                    status_right = EYE_CLASS_LABELS[1] if prob_right > 0.5 else EYE_CLASS_LABELS[0]
                    color = (0, 255, 0) if status_right == 'Mata Terbuka' else (0, 0, 255)
                    cv2.rectangle(frame, (rx_min-padding, ry_min-padding), (rx_max+padding, ry_max+padding), color, 2)
                    cv2.putText(frame, status_right, (rx_min-padding, ry_min-15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Cek kondisi mata tertutup
            if status_left == 'Mata Tertutup' and status_right == 'Mata Tertutup':
                eyes_closed_counter += 1
            else:
                eyes_closed_counter = 0
            
            if eyes_closed_counter >= EYES_CLOSED_FRAMES_THRESHOLD:
                drowsiness_alert = True
                alert_reason = "Mata Tertutup"

            # --- C. Analisis Menguap ---
            current_mar = calculate_mar(face_landmarks)
            mar_buffer.append(current_mar)

            if len(mar_buffer) == MAR_WINDOW_SIZE:
                features = np.array([[np.mean(mar_buffer), np.max(mar_buffer), np.std(mar_buffer)]])
                scaled_features = YAWN_SCALER.transform(features)
                prediction = YAWN_SVM_MODEL.predict(scaled_features)[0]
                yawn_status = "MENGUAP" if prediction == 1 else "NORMAL"
            
            # Cek kondisi menguap
            if yawn_status == "MENGUAP":
                drowsiness_alert = True
                alert_reason = "Menguap"

            # Visualisasi Mulut
            mouth_x = [face_landmarks.landmark[i].x * w for i in MOUTH_INDICES]
            mouth_y = [face_landmarks.landmark[i].y * h for i in MOUTH_INDICES]
            mouth_x1, mouth_y1 = int(min(mouth_x)), int(min(mouth_y))
            mouth_x2, mouth_y2 = int(max(mouth_x)), int(max(mouth_y))
            cv2.rectangle(frame, (mouth_x1, mouth_y1), (mouth_x2, mouth_y2), (255, 0, 0), 1)
            cv2.putText(frame, f"Mulut: {yawn_status}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame, f"MAR: {current_mar:.2f}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            # --- D. Visualisasi Wajah ---
            all_x = [lm.x * w for lm in face_landmarks.landmark]
            all_y = [lm.y * h for lm in face_landmarks.landmark]
            face_x1, face_y1 = int(min(all_x)), int(min(all_y))
            face_x2, face_y2 = int(max(all_x)), int(max(all_y))
            cv2.rectangle(frame, (face_x1, face_y1), (face_x2, face_y2), (0, 255, 0), 2)
            
        # Tampilkan Peringatan jika terdeteksi kantuk
        if drowsiness_alert:
            cv2.putText(frame, "!!! PERINGATAN KANTUK !!!", (int(w/2) - 250, int(h/2) - 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3, cv2.LINE_AA)
            cv2.putText(frame, f"Penyebab: {alert_reason}", (int(w/2) - 120, int(h/2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        cv2.imshow('Sistem Deteksi Kantuk Terintegrasi', frame)
        if cv2.waitKey(5) & 0xFF == ord('q'): break

    cap.release()
    cv2.destroyAllWindows()
    face_mesh.close()
    print("Aplikasi ditutup.")

if __name__ == '__main__':
    main()

✅ Model Head Pose berhasil dimuat.
✅ Model Status Mata berhasil dimuat.
✅ Model Deteksi Menguap (SVM) berhasil dimuat.
🎥 Webcam berhasil dibuka. Tekan 'q' untuk keluar.


c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not ha

Aplikasi ditutup.
